In [ ]:
# #import install
!pip install tweepy
!pip install seaborn
!pip install textblob
!pip install nltk
!pip install jsonpickle
#libraries and packages

In [ ]:

import tweepy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
from nltk import word_tokenize, pos_tag
from nltk.corpus import sentiwordnet as swn
from nltk.wsd import lesk

from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter
import nltk
import string
import time
import jsonpickle


# Data Preparation

In [ ]:

# Consume:
CONSUMER_KEY = 'SS97NM79TEdKPsSWlmnhUcOwd'
CONSUMER_SECRET = 'rZpFgeEMsCp6cYZ4u2gSTVj9sFXQUMVh5lVkDAEV2b3MTEGyE2'

# Access:
ACCESS_TOKEN = '1037289423321350144-aG0QZqLs0cOHIuwNgagYIRTZVT40b4'
ACCESS_SECRET = 'O4qSyIY74M7iUqQB2Qgb92H8AwIvCV5O3SjSA8rJFbc10'


def connect_to_twitter_OAuth():
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    api = tweepy.API(auth,wait_on_rate_limit=True)
    return api


# Create API object
api = connect_to_twitter_OAuth()

### Extracting the tweets

In [ ]:
"""# Extracting the tweets"""

def get_save_tweets(filepath, api, query,max_tweets=1000, lang='en'):
    tweetCount = 0

    # Open file and save tweets
    with open(filepath, 'w') as f:
        # Send the query
        for tweet in tweepy.Cursor(api.search_tweets, q=query, lang=lang, since_id='2017-01-01').items(max_tweets):
        # for tweet in tweepy.Cursor(api.search_tweets, q=query, lang=lang).items(max_tweets):
            # Convert to JSON format
            f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
            tweetCount += 1

        # Display how many tweets we have collected
        print("Downloaded {0} tweets".format(tweetCount))
query = 'airlines'# change the query topic Ex:'BuzzFeed' for Business tweets 
get_save_tweets('Airlines.json', api, query)# change the filename ".json" for different dataset


### Create the dataframe for extracted tweets

In [ ]:
"""# Create the dataframe for extracted tweets"""

def tweets_to_df(path):
    tweets = list(open(path, 'rt'))

    text = []
    weekday = []
    month = []
    day = []
    hour = []
    user = []
    

    for t in tweets:
        t = jsonpickle.decode(t)

        # Text
        text.append(t['text'])
        # Add user
        user.append(t['user']['name'])

        

    d = {'text': text,
                
         'user': user      
         }

    return pd.DataFrame(data=d)

tweets_df =  tweets_to_df("Airlines.json")
tweets_df.head()

### Clean the text and classify the polarity of a tweet

In [ ]:
from textblob import TextBlob
import re


def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())


def analize_sentiment(tweet):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return 2

In [ ]:
tweets_df['SA'] = np.array([analize_sentiment(text) for text in tweets_df['text']])


pos_tweets = [ text for index, text in enumerate(tweets_df['text']) if tweets_df['SA'][index] == 1]
neu_tweets = [ text for index, text in enumerate(tweets_df['text']) if tweets_df['SA'][index] == 0]
neg_tweets = [ text for index, text in enumerate(tweets_df['text']) if tweets_df['SA'][index] == 2]


In [ ]:
tweets_df

# Data Preprocesing

In [ ]:
custom_stop_words = ['i','me', 'my','myself','we','our','ours','ourselves','you',"you're","you've","you'll","you'd",'your','yours','yourself','yourselves','he','him','his','himself','she',"she's",'her','hers','herself','it',"it's",'its','itself','they','them','their','theirs','themselves','what','which','who','whom','this','that',"that'll",'these','those','am','is','are','was','were','be','been','being','have','has','had','having','do','does','did','doing','a','an','the','and','but','if','or','because','as','until','while','of','at','by','for','then','once','here','there','when','where','why','how','all','any','both','no','nor','not','only','own','same','so','s','t','can','will','just','don',"don't",'should',"should've",'now','d','ll','m','o','re','ve','y','ain','aren',"aren't",'couldn',"couldn't",'didn',"didn't",'doesn',"doesn't",'hadn',"hadn't",'hasn',"hasn't",'haven',"haven't",'isn',"isn't",'ma','mightn',"mightn't",'mustn',"mustn't",'needn',"needn't",'shan',"shan't",'shouldn',"shouldn't",'wasn',"wasn't",'weren',"weren't",'won',"won't",'wouldn',"wouldn't","to"]
stop_words = set(custom_stop_words)
nltk.download('punkt')
def tokenizer(s):
    s = re.sub(r'http\S+', '', s)
    s = re.sub(r'\$(\w+)','',s)
    translate_table = dict((ord(char), None) for char in string.punctuation)   
    s = s.translate(translate_table)
    tokens = nltk.word_tokenize(s)
    filtered_tokens =[]
    for word in tokens:
        if word.lower() not in stop_words:
            filtered_tokens.append(word.lower())
    return filtered_tokens

# ['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']

tweets_df['TweetText'] = tweets_df['text'].apply(tokenizer)
tweets_df['text_length'] = tweets_df['text'].apply(len)
tweets_df = tweets_df[(tweets_df['text_length']>0)]


In [ ]:
from itertools import chain
text = np.array(list(chain(*tweets_df['text'])))

In [ ]:
words= []
for text in tweets_df.TweetText:
    for word in text:
        words.append(word)

from collections import Counter
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
#len(vocab_to_int)

# Feature Extraction

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Use tf-idf features

tfidf_vectorizer = TfidfVectorizer()

tfidf = tfidf_vectorizer.fit_transform(text)

# Use tf features
tf_vectorizer = CountVectorizer()
tf = tf_vectorizer.fit_transform(text)

tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
print("Number of total features: {}".format(len(tfidf_feature_names)))

In [ ]:
feature_names = np.array(tfidf_feature_names)
feature_names

In [ ]:
# Initialize Ida
from collections import Counter
from sklearn.decomposition import NMF, LatentDirichletAllocation
lda = LatentDirichletAllocation(max_iter=10,
                                learning_method='online',
                                learning_offset=50.,
                                n_components=10,
                                random_state=0)

In [ ]:
lda_tfidf = lda.fit(tfidf)

In [ ]:
lda_W = lda_tfidf.transform(tfidf)
Counter([np.argmax(i) for i in lda_W])

In [ ]:
max_len = 50


def fun(x):
    vec = []
    zeroes = max_len - len(x)
    if zeroes>0:
        for i in range(zeroes):
            vec.append(0)
    
    for i in range(max_len-len(vec)):
        word = x[i]
        vec.append(vocab_to_int[word])
    return np.array(vec)

tweets_df['encoded_text'] = tweets_df['TweetText'].apply(fun)

tweets_df.head()

In [ ]:
num_top_words = 15
tp = np.zeros(shape=[0,5])

def retrieve_top_words(model, feature_names, num_top_words):
    for idx, topic in enumerate(model.components_):
        print("Topic #{}:".format(idx),topic, end='\n')
        tp = topic[:-num_top_words - 1:-1]
        print(tp)
    imp_feat = []
    for vec in tweets_df.encoded_text:
        imp_feat.append(vec.tolist())
    imp_feat =np.array(imp_feat)
    return imp_feat
    

In [ ]:
features = retrieve_top_words(lda_tfidf, tfidf_feature_names, num_top_words)

In [ ]:
p = len(pos_tweets)*100/len(tweets_df['TweetText'])
n = len(neg_tweets)*100/len(tweets_df['TweetText'])
nu = len(neu_tweets)*100/len(tweets_df['TweetText'])
#print(tweets_df.head(10))
print("Percentage of positive tweets: {}%".format(p))
print("Percentage of neutral tweets: {}%".format(nu))
print("Percentage de negative tweets: {}%".format(n))

colors = ['green', 'red', 'grey']
sizes = [p,n,nu]
labels = 'Positive', 'Negative', 'Neutral'

## use matplotlib to plot the chart
plt.pie(
   x=sizes,
   shadow=True,
   colors=colors,
   labels=labels,
   startangle=90
)

plt.title("Sentiment Analysis.")
plt.show()

In [ ]:
from __future__ import print_function
import numpy as np
# !pip install PyPrind
from keras.layers import Embedding,Dropout, LSTM,Bidirectional
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import keras
from keras.layers import *
# from keras.layers.embeddings import Embedding
from keras.models import Sequential
# from keras_self_attention import SeqSelfAttention
# import pyprind
#max_features = 15000
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 500
batch_size = 64
max_num_words = 20000
max_length = 500
print('Loading data...')
x = features
y = tweets_df['SA'].values 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33) 

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
# from EMARectified_adam import EMARectifiedAdam
# # tf.gather(list, tf_look_up[index])
# optm = EMARectifiedAdam(lr=1e-3)
model = Sequential()
model.add(Embedding(max_num_words, 100, input_length=max_length))
# model.add(SeqSelfAttention(attention_activation='relu'))
model.add(Bidirectional(LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
model.add(Bidirectional(LSTM(units=64, return_sequences=False, dropout=0.2)))
model.add(Dense(3, activation='softmax'))
model.compile('adam', 'sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3, verbose=2)


In [ ]:
y_pred = model.predict(x_test)
y_pred

In [ ]:
lstm_pred=np.argmax(y_pred,axis=1)

In [ ]:
model.summary()

In [ ]:
from sklearn import metrics
target_names = ['Neutral', 'Positive', 'Negative']
print("ACCURACY is:",metrics.accuracy_score(y_test, lstm_pred)) 
print("Confusion Matrix",metrics.confusion_matrix(y_test, lstm_pred))
print("Classification_Report is:\n",metrics.classification_report(y_test, lstm_pred, target_names=None))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, lstm_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, lstm_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, lstm_pred))) 

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics

# actual = numpy.random.binomial(1,.9,size = 1000)
# predicted = numpy.random.binomial(1,.9,size = 1000)

# confusion_matrix = metrics.confusion_matrix(actual, predicted)
confusion_matrix = metrics.confusion_matrix(y_test, lstm_pred)


cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix)

cm_display.plot()
plt.show()

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=30).fit(x_train, y_train)
mlp_pred=clf.predict(x_test)
print("ACCURACY is:",metrics.accuracy_score(y_test, mlp_pred)) 
print("Confusion Matrix",metrics.confusion_matrix(y_test, mlp_pred))
print("Classification_Report is:\n",metrics.classification_report(y_test, mlp_pred, target_names=target_names))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, mlp_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, mlp_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, mlp_pred))) 

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test, mlp_pred)


cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix)

cm_display.plot()
plt.show()

In [ ]:
from sklearn.svm import SVC
svm_clf = SVC(C=3,random_state=0).fit(x_train, y_train)
svm_pred=svm_clf.predict(x_test)
print("ACCURACY is:",metrics.accuracy_score(y_test, svm_pred)) 
print("Confusion Matrix",metrics.confusion_matrix(y_test, svm_pred))
print("Classification_Report is:\n",metrics.classification_report(y_test, svm_pred, target_names=target_names))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, svm_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, svm_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, svm_pred))) 

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics

# actual = numpy.random.binomial(1,.9,size = 1000)
# predicted = numpy.random.binomial(1,.9,size = 1000)

# confusion_matrix = metrics.confusion_matrix(actual, predicted)
confusion_matrix = metrics.confusion_matrix(y_test, svm_pred)


cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix)

cm_display.plot()
plt.show()